## Introduction to the SAMPLED BASED DIAGONALIZATION

In [ ]:
from src.hamiltonian_utils import FermiHubbardHamiltonian
from src.nuclear_physics_utils import get_twobody_nuclearshell_model,SingleParticleState,QuadrupoleOperator,J2operator
import numpy as np
import torch
from typing import Dict
import scipy
from src.qml_models import AdaptVQEFermiHubbard
from src.qml_utils.train import Fit
from src.qml_utils.utils import configuration
from scipy.sparse.linalg import eigsh,expm_multiply
from tqdm import trange
import matplotlib.pyplot as plt
from src.utils_quasiparticle_approximation import QuasiParticlesConverterOnlynnpp,QuasiParticlesConverter
from scipy.sparse import lil_matrix
file_name='data/usdb.nat'

SPS=SingleParticleState(file_name=file_name)

In [ ]:
nparticles_a=6
nparticles_b=6

size_a=SPS.energies.shape[0]//2
size_b=SPS.energies.shape[0]//2

title=r'$^{18}$O'


In [ ]:

# Compute the J^2 value
#J2Class=J2operator(size_a=size_a,size_b=size_b,nparticles_a=nparticles_a,nparticles_b=nparticles_b,single_particle_states=SPS.state_encoding,j_square_filename=file_name+'_j2',symmetries=[SPS.total_M_zero])

#Quadrupole Operator


# compute the NSM Hamiltonian
NSMHamiltonian=FermiHubbardHamiltonian(size_a=size_a,size_b=size_b,nparticles_a=nparticles_a,nparticles_b=nparticles_b,symmetries=[SPS.total_M_zero])
print('size=',size_a+size_b,size_b)
NSMHamiltonian.get_external_potential(external_potential=SPS.energies[:size_a+size_b])
if file_name=='data/cki':
    twobody_matrix,energies=get_twobody_nuclearshell_model(file_name=file_name)

    NSMHamiltonian.get_twobody_interaction(twobody_dict=twobody_matrix)
else:
    NSMHamiltonian.twobody_operator=scipy.sparse.load_npz(f'data/nuclear_twobody_matrix/usdb_{nparticles_a}_{nparticles_b}.npz')
NSMHamiltonian.get_hamiltonian()

egs,psi0=NSMHamiltonian.get_spectrum(n_states=1)

print(egs)

print('total_m=',SPS.compute_m_exp_value(psi=psi0,basis=NSMHamiltonian.basis))
#print('j_value=',J2Class.j_value(psi0))
print('dimension=',NSMHamiltonian.hamiltonian.shape[0])

#### Select the samples

In [ ]:
prob=np.conjugate(psi0[:,0])*psi0[:,0]

In [ ]:

n_instances=400
selection=np.random.choice(np.arange(prob.shape[0]),size=n_instances,replace=False,p=prob)
print(selection)
effective_hamiltonian=lil_matrix((n_instances,n_instances))

for a,idx_a in enumerate(selection):
    for b,idx_b in enumerate(selection):
        effective_hamiltonian[a,b]=NSMHamiltonian.hamiltonian[idx_a,idx_b]


effective_egs,effective_psi0=eigsh(effective_hamiltonian,k=1,which='SA')

print(effective_egs)


In [ ]:
plt.figure(figsize=(10,10))
plt.ylabel('Counts',fontsize=30)
plt.hist(prob,bins=1000)
plt.xlabel(r'$p$',fontsize=30)
plt.tick_params(labelsize=30)
plt.show()



# Compute histogram (don't plot yet)
counts, bin_edges = np.histogram(prob, bins=2000,density=False)

# Bin centers (x-axis)
bin_centers = 0.5 * (bin_edges[1:] + bin_edges[:-1])

# Filter out zero counts to avoid log(0)
nonzero = counts > 0
counts = counts[nonzero]

print(counts)
bin_centers = bin_centers[nonzero]
bin_widths = np.diff(bin_edges)

pdf = counts #/ (np.sum(counts))

# Plot in log-log scale
plt.figure(figsize=(8,6))
plt.loglog(bin_centers, pdf, marker='o', linestyle='none')
plt.xlabel(r'$p$', fontsize=20)
plt.ylabel('Counts', fontsize=20)
plt.tick_params(labelsize=15)
plt.title(r'Tail of Histogram (Log-Log) in $^{24}$Mg', fontsize=18)
plt.grid(True, which="both", ls="--")
plt.tight_layout()
plt.show()

In [ ]:
from scipy import stats
mask = counts > 0
x = bin_centers[mask]
y = counts[mask]

# Take logs
logx = np.log(x[:-1])
logy = np.log(y[:-1])

# Fit linear regression in log-log space
slope, intercept, r_value, p_value, std_err = stats.linregress(logx, logy)

# Extract power-law parameters
b = -slope
a = np.exp(intercept)
print(intercept)
print(f"Fitted power law: y ≈ {a:.6f} * x^(-{b:.3f})")
print(f"R² = {r_value**2:.4f}")

# Plot
plt.figure(figsize=(8,6))

plt.title(r'Tail of the histogram in $^{28}$Si', fontsize=18)
plt.loglog(x, y, 'o', label='Histogram data')
plt.loglog(x, a * x**(-b), 'r--', label=f'Fit: y ∝ exp^({np.log(a):.2f}) x^(-{b:.2f})')
plt.xlabel(r'$p$', fontsize=20)
plt.ylabel('Counts', fontsize=20)
plt.tick_params(labelsize=15)
plt.legend()
plt.grid(True, which="both", ls="--")
plt.tight_layout()
plt.show()

#### Build-up an algorithm to save the number of instances with a fixed accuracy

Hyperparameters

In [ ]:
nparts=[(2,0),(4,0),(6,0),(8,0),(2,2),(4,2),(6,2),(8,2),(4,4),(6,4),(8,4),(6,6),(8,6),(10,6),(10,8)]
titles=[r'$^{18}$O',r'$^{20}$O',r'$^{22}$O',r'$^{24}$O',r'$^{20}$Ne',r'$^{22}$Ne',r'$^{24}$Ne',r'$^{26}$Ne',r'$^{24}$Mg',r'$^{26}$Mg',r'$^{28}$Mg',r'$^{28}$Si',r'$^{30}$Si',r'$^{32}$Si',r'$^{32}$Ar']

history={}
errors=[]
n_instances_max=[]

for r,title in enumerate(titles):
    print('title=',title)
    nparticles_a=nparts[r][0]
    nparticles_b=nparts[r][1]
    
    history[nparts[r]]=[]
    
    NSMHamiltonian=FermiHubbardHamiltonian(size_a=size_a,size_b=size_b,nparticles_a=nparticles_a,nparticles_b=nparticles_b,symmetries=[SPS.total_M_zero])
    print('size=',size_a+size_b,size_b)
    NSMHamiltonian.get_external_potential(external_potential=SPS.energies[:size_a+size_b])
    if file_name=='data/cki':
        twobody_matrix,energies=get_twobody_nuclearshell_model(file_name=file_name)

        NSMHamiltonian.get_twobody_interaction(twobody_dict=twobody_matrix)
    else:
        NSMHamiltonian.twobody_operator=scipy.sparse.load_npz(f'data/nuclear_twobody_matrix/usdb_{nparticles_a}_{nparticles_b}.npz')
    NSMHamiltonian.get_hamiltonian()

    egs,psi0=NSMHamiltonian.get_spectrum(n_states=1)

    print(egs)

    print('total_m=',SPS.compute_m_exp_value(psi=psi0,basis=NSMHamiltonian.basis))
    #print('j_value=',J2Class.j_value(psi0))
    print('dimension=',NSMHamiltonian.hamiltonian.shape[0])
    
    prob=np.conjugate(psi0[:,0])*psi0[:,0]
    
    n_instances=np.arange(2,prob.shape[0])
    
    error=1000
    i=0
    while(error> 10**-3):
        selection=np.random.choice(np.arange(prob.shape[0]),size=n_instances[i],replace=False,p=prob)
        # effective_hamiltonian=lil_matrix((n_instances[i],n_instances[i]))

        # for a,idx_a in enumerate(selection):
        #     for b,idx_b in enumerate(selection):
        #         effective_hamiltonian[a,b]=NSMHamiltonian.hamiltonian[idx_a,idx_b]
        
        effective_hamiltonian=  NSMHamiltonian.hamiltonian.tocsr()[selection,:][:,selection]
        
        effective_egs,effective_psi0=eigsh(effective_hamiltonian,k=1,which='SA')
        error=np.abs(effective_egs[0]-egs[0])/np.abs(egs[0])
        history[nparts[r]].append(error)
        
        i+=1
    
    n_instances_max.append(i)
    errors.append(error)
    print('\n')
    print(n_instances[i])
    print('error=',np.abs(effective_egs[0]-egs[0])/np.abs(egs[0]),'\n')     
    
    

#### Analysis of the results

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

data=np.load('data/samplebaseddiagonalization_data/run_1%.npz')

errors=data['errors']
titles=data['titles']
history=data['history']
n_instances=data['n_instances']
dimensions=data['dimensions']

print(titles)
# 0 5 9 12
selection=[0,5,9,12]
print(titles[selection])

In [ ]:
plt.figure(figsize=(20,10))
plt.title('SBD- Nuclear Shell model Hamiltonian in the sd-shell',fontsize=30)
plt.ylabel('n instances',fontsize=30)
plt.bar(np.arange(n_instances.shape[0]),dimensions)
plt.bar(np.arange(n_instances.shape[0]),n_instances)
#plt.semilogy()
plt.xticks(np.arange(n_instances.shape[0]),titles[:n_instances.shape[0]])
#plt.text(x=1,y=80000,s=r'$H=\sum_a e_a c^{+}_a c_a +\frac{1}{4}\sum_{abcd} v_{abdc} c^{+}_a c^{+}_b c_c c_d$',fontsize=30)
plt.tick_params(labelsize=20)
plt.show()

In [ ]:
plt.figure(figsize=(20,10))
plt.title('SBD- Nuclear Shell model Hamiltonian in the sd-shell (1% accuracy energy)',fontsize=30)
plt.ylabel(r'$N_s$',fontsize=30)
plt.xlabel(r'$\dim$',fontsize=30)

plt.scatter(dimensions,dimensions)
plt.plot(dimensions,dimensions)
plt.scatter(dimensions,n_instances)
plt.loglog()

plt.tick_params(labelsize=20)
plt.show()

In [ ]:
plt.scatter(n_instances,dimensions)
plt.loglog()
plt.show()

from scipy import stats

# Log-transform
x_tilde = np.log(dimensions)
y_tilde = np.log(n_instances)

# Linear regression
slope, intercept, r_value, p_value, std_err = stats.linregress(x_tilde, y_tilde)
b = slope
a = np.exp(intercept)

print(f"Fitted model: y ≈ {a:.3f} * x^{b:.3f}")
print(f"R² = {r_value**2:.4f}")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

# Assuming these are your data
# dimensions = ...
# n_instances = ...

# Log-transform
x_tilde = np.log(dimensions)
y_tilde = np.log(n_instances)

# Linear regression
slope, intercept, r_value, p_value, std_err = stats.linregress(x_tilde, y_tilde)
b = slope
a = np.exp(intercept)

print(f"Fitted model: y ≈ {a:.3f} * x^{b:.3f}")
print(f"R² = {r_value**2:.4f}")

# Plot original data and the fitted curve
plt.figure(figsize=(8,6))
plt.scatter(dimensions, n_instances, label='Data', color='blue')

# Create smooth line for fitted curve
x_fit = np.linspace(min(dimensions), max(dimensions), 200)
y_fit = a * x_fit**b
plt.plot(x_fit, y_fit, label=f'Fit: y ≈ {a:.2f} * x^{b:.2f}', color='red', linestyle='--')

# Log scales
plt.xscale('log')
plt.yscale('log')

# Labels
plt.xlabel('Dimension', fontsize=14)
plt.ylabel('Number of Instances', fontsize=14)
plt.legend()
plt.grid(True, which="both", ls="--", lw=0.5)
plt.tick_params(labelsize=20)
plt.tight_layout()
plt.show()
